# Identificando os outliers


###### Outliers são pontos de dados se desviam de forma acentuada dos outros pontos de dados em uma mesma amostra. Isso pode ocorrer em função de características particulares que afetam a variável analisada, erro na transcrição/registo dos valores da variável, etc. A visualização gráfica desses valores em boxplot auxilia a compreender o quanto eles estão distante do valor esperado.


In [ ]:
dtview.check_outliers_univariate(df_274_time_sale)

In [ ]:
df_sem_outliers = df_274_time_sale[df_274_time_sale['y'] < 500]
df_sem_outliers.info

In [ ]:
df_274_sem_outliers_treino, df_274_sem_outliers_teste = dtexp.configura_dataframe_treino_teste(df_sem_outliers)
periods_outliers = df_274_sem_outliers_teste.shape[0]


In [ ]:
iplot(dtview.compare_dataframes_with_scatter(df_274_sem_outliers_treino,
                                   df_274_sem_outliers_teste,
                                   title='Montante de dados de vendas de Treinamento e Teste (sem outliers)',
                                    mode='lines',
                                   name1='Treinamento',
                                   name2='Teste'))

In [ ]:
prophet_no_outliers = Prophet()
prophet_no_outliers.fit(df_274_sem_outliers_treino)
future_no_outliers = prophet_no_outliers.make_future_dataframe(periods=periods_outliers, freq='30min')
# Removendo valores fora do range de atendimento da loja (22:30 - 05:59)
future_no_outliers['ds'] = pd.to_datetime(future_no_outliers['ds'])
future_no_outliers = future_no_outliers.set_index(pd.DatetimeIndex(future_no_outliers['ds']))
future_no_outliers = future_no_outliers.between_time('08:00','21:00')

forecast_no_outliers = prophet_no_outliers.predict(future_no_outliers)
forecast_no_outliers = dtclean.remove_negative_values(forecast_no_outliers)
iplot(dtview.plot_forecast_as_table(forecast_no_outliers))

In [ ]:
# componentes

fig = prophet_no_outliers.plot_components(forecast_no_outliers, figsize=(12,8))


# Visualizando a previsão

In [ ]:
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
fig = prophet_no_outliers.plot(forecast_no_outliers,ax=ax)
plt.show()

In [ ]:
df_274_no_outliers_teste_forecast = prophet_no_outliers.predict(df_274_sem_outliers_teste)
df_274_no_outliers_teste_forecast = dtclean.remove_negative_values(df_274_no_outliers_teste_forecast)

In [ ]:
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
ax.scatter(df_274_sem_outliers_teste.ds,df_274_sem_outliers_teste['y'], color='r')
fig = prophet_no_outliers.plot(df_274_no_outliers_teste_forecast,ax=ax)
plt.show()

# Comparando os dados previstos com os atuais

In [ ]:
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
ax.scatter(df_274_sem_outliers_teste.ds, df_274_sem_outliers_teste['y'], color='r')
fig = prophet_no_outliers.plot(df_274_no_outliers_teste_forecast, ax=ax)
ax.set_xbound(lower='2020-05-24', upper='2020-05-31')
ax.set_ylim(0, 1000)
plot = plt.suptitle('Última semana de Maio - Previsão vs Original')

In [ ]:
iplot(dtview.compare_dataframes_with_scatter(df_274_sem_outliers_teste,
                                   df_274_no_outliers_teste_forecast,
                                   title='Previsão de vendas vs Vendas originais realizadas',
                                    mode='lines',
                                   name1='Teste',
                                   name2='Previsto',
                                  is_forecast=True))

In [ ]:
iplot(dtview.compare_dataframes_with_bar(
        df_274_sem_outliers_teste,
        df_274_no_outliers_teste_forecast,
        nametrace1='Valor Real (sem outliers)',
        nametrace2='Valor Previsto (sem outliers)',
        idx_end=28))

In [ ]:
print("MAPE ---> {}".format(
    fm.mape(np.array(df_274_sem_outliers_teste['y']),
            np.array(df_274_no_outliers_teste_forecast['yhat']))))

## Adicionando feriados ao modelo

In [ ]:
prophet_feriados_s_outliers = Prophet(holidays=dtclean.get_Holiday())

prophet_feriados_s_outliers.add_country_holidays('BR')
prophet_feriados_s_outliers.fit(df_274_sem_outliers_treino)
# Criando as datas futuras previstas pelo prophet
# A frequência especificada em horas (os dados estão em horas)
future_feriados_s_outliers = prophet_feriados_s_outliers.make_future_dataframe(freq='30min', periods=periods_outliers)
# Prevendo os valores
forecast_feriados_s_outliers = prophet_feriados_s_outliers.predict(future_feriados_s_outliers)
# Removendo valores fora do range de atendimento da loja (22:30 - 05:59)
future_feriados_s_outliers['ds'] = pd.to_datetime(future_feriados_s_outliers['ds'])
future_feriados_s_outliers = future_feriados_s_outliers.set_index(pd.DatetimeIndex(future_feriados_s_outliers['ds']))
future_feriados_s_outliers = future_feriados_s_outliers.between_time('08:00','21:00')
# Previsão optimizada dos valores
future_feriados_s_outliers = prophet_feriados_s_outliers.predict(future_feriados_s_outliers)
future_feriados_s_outliers = dtclean.remove_negative_values(future_feriados_s_outliers)

In [ ]:
plt,fig_forecast_feriado_s_outliers = dtview.plot_forecast(future_feriados_s_outliers,prophet_feriados_s_outliers,f,ax)
plt.show()

# Visualizando os componentes do modelo

In [ ]:
fig_forecast_feriado_s_outliers = prophet_feriados_s_outliers.plot_components(forecast_feriados_s_outliers,figsize=(12,8))

In [ ]:
forecast_teste_feriados_s_outliers = prophet_feriados_s_outliers.predict(df_274_sem_outliers_teste)
forecast_teste_feriados_s_outliers = dtclean.remove_negative_values(forecast_teste_feriados_s_outliers)

# Comparando os dados previsto e original com feriados

In [ ]:
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
ax.scatter(df_274_sem_outliers_teste.ds, df_274_sem_outliers_teste['y'], color='r')
fig = prophet_feriados_s_outliers.plot(forecast_teste_feriados_s_outliers, ax=ax)
ax.set_xbound(lower='2020-05-24', upper='2020-05-31')
ax.set_ylim(0, 1000)
plot = plt.suptitle('Última semana de Maio - Previsão vs Original (com feriados)')

In [ ]:
iplot(dtview.compare_dataframes_with_scatter(
    df_274_sem_outliers_teste,
    forecast_teste_feriados_s_outliers,
    title='Dados Previsto vs Original última semana de maio/2020',
    mode='lines',
    name1='Original',
    name2='Previsão com feriados',
    is_forecast=True))

In [ ]:
iplot(dtview.compare_dataframes_with_bar(
        df_274_sem_outliers_teste,
        forecast_teste_feriados_s_outliers,
        nametrace1='Original',
        nametrace2='Previsão com feriados',
        idx_end=28))

In [ ]:
print("MAPE ---> {}".format(
    fm.mape(np.array(df_274_sem_outliers_teste['y']),
    np.array(forecast_teste_feriados_s_outliers['yhat']))))


# Configurando e treinando o modelo com feriados e parâmetros otimizados


In [ ]:
final_prophet_s_outliers = Prophet(holidays=dtclean.get_Holiday(),
                      changepoint_prior_scale= 15,
                      holidays_prior_scale = 15,
                      n_changepoints = 200,
                      seasonality_mode = 'multiplicative',
                      seasonality_prior_scale = 25,
                      weekly_seasonality=True,
                      daily_seasonality = True,
                      yearly_seasonality = False,
                      interval_width=0.95)
final_prophet_s_outliers.add_country_holidays(country_name='BR')
final_prophet_s_outliers.fit(df_274_sem_outliers_treino)

In [ ]:
future_final_s_outliers = final_prophet_s_outliers.make_future_dataframe(periods=periods_outliers, freq='30min')
# Removendo valores fora do range de atendimento da loja (22:30 - 05:59)
future_final_s_outliers['ds'] = pd.to_datetime(future_final_s_outliers['ds'])
future_final_s_outliers = future_final_s_outliers.set_index(pd.DatetimeIndex(future_final_s_outliers['ds']))
future_final_s_outliers = future_final_s_outliers.between_time('08:00','21:00')
forecast_final_s_outliers = final_prophet_s_outliers.predict(future_final_s_outliers)
forecast_final_s_outliers = dtclean.remove_negative_values(forecast_final_s_outliers)
iplot(dtview.plot_forecast_as_table(forecast_final_s_outliers))


In [ ]:
# Visualizando os componentes do modelo
fig = final_prophet_s_outliers.plot_components(forecast_final_s_outliers,figsize=(12,8))

In [ ]:
# Visualizando a previsão

f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
fig = final_prophet_s_outliers.plot(forecast_final_s_outliers,ax=ax)
plt.show()

In [ ]:
df_teste_final_s_outliers= final_prophet_s_outliers.predict(df_274_sem_outliers_teste)
df_teste_final_s_outliers = dtclean.remove_negative_values(df_teste_final_s_outliers)



# Comparando os dados reais com os dados previstos

In [ ]:
f, ax = plt.subplots(1)
f.set_figheight(5)
f.set_figwidth(15)
ax.scatter(df_274_sem_outliers_teste.ds, df_274_sem_outliers_teste['y'], color='r')
fig = final_prophet_s_outliers.plot(df_teste_final_s_outliers, ax=ax)
ax.set_xbound(lower='2020-05-24', upper='2020-05-31')
ax.set_ylim(0, 1000)
plot = plt.suptitle('Última semana de Maio - Previsão vs Original (com parametros)')


In [ ]:
iplot(dtview.compare_dataframes_with_scatter(df_274_sem_outliers_teste,
                                   df_teste_final_s_outliers,
                                   title='Previsão de vendas vs Vendas originais realizadas',
                                    mode='lines',
                                   name1='Teste',
                                   name2='Previsto',
                                  is_forecast=True))

In [ ]:
iplot(dtview.compare_dataframes_with_bar(
    df_274_sem_outliers_teste,
    df_teste_final_s_outliers,
    nametrace1='Teste',
    nametrace2='Previsto'))

In [ ]:
print("MAPE (validação) ---> {}".format(
    fm.mape(np.array(df_274_sem_outliers_teste['y']),
    np.array(df_teste_final_s_outliers['yhat']))))